In [2]:
pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
# from google.colab import drive
import re
import string
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
from transformers import TrainingArguments

# Genre Dataset

In [2]:
train_dataset = pd.read_csv("/Users/marianivethaantonypushparaj/Desktop/nlp/Genre-Final/train_dataset_truncated.csv")
test_dataset = pd.read_csv("/Users/marianivethaantonypushparaj/Desktop/nlp/Genre-Final/test_genre.csv")

# Data Exploration

In [4]:
train_dataset

,Unnamed: 0,Artist,Song,Genre,Language,Lyrics
0,0,brad paisley,be the lake,Country,en,there aint nothing that i would rather see tha...
1,1,gene watson,sometimes i get lucky and forget,Country,en,i stay out late most every night until closing...
2,2,charlie landsborough,long and heavy chain,Country,en,your hunger after money your parents agree the...
3,3,be good tanyas,rain and snow,Country,en,well i married me a wife she gave me trouble a...
4,4,buck owens,ashes of love,Country,en,ashes of love cold as ice you made the debt il...
...,...,...,...,...,...,...
44804,44804,echo & the bunnymen,evergreen,Rock,en,theres no more wishes in the well no more drea...
44805,44805,journey,"mother, father",Rock,en,she sits alone an empty stare a mothers face s...
44806,44806,xtc,science friction,Rock,en,i look out of my window at night i see the sta...
44807,44807,wilco,art of almost,Rock,en,i cant be so far away from my wasteland ill ne...


In [5]:
train_dataset['Language'].unique()

array(['en'], dtype=object)

In [ ]:
train_dataset['Genre'].unique()

array(['Rock', 'Metal', 'Pop', 'Indie', 'R&B', 'Folk', 'Electronic',
       'Jazz', 'Hip-Hop', 'Country'], dtype=object)

In [6]:
train_dataset = train_dataset[train_dataset['Language'] == 'en']

In [7]:
train_dataset.drop_duplicates(subset=['Lyrics'], inplace=True)
test_dataset.drop_duplicates(subset=['Lyrics'],inplace=True)

# Data Cleaning

In [8]:
train_dataset['Lyrics'] = train_dataset['Lyrics'].str.replace('\n', ' ').str.lower()
test_dataset['Lyrics'] = test_dataset['Lyrics'].str.replace('\n', ' ').str.lower()

In [9]:
def remove_punctuation(input_string):
    translator = str.maketrans("", "", string.punctuation)
    result = input_string.translate(translator)
    return result

In [10]:
for i in range(0,len(train_dataset)):
    train_dataset['Lyrics'].iloc[i] = remove_punctuation(train_dataset['Lyrics'].iloc[i])

/var/folders/28/gj9cb4w95rb4sbjdp44jc_hh0000gn/T/ipykernel_20245/623218609.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Lyrics'].iloc[i] = remove_punctuation(train_dataset['Lyrics'].iloc[i])


In [11]:
for i in range(0,len(test_dataset)):
    test_dataset['Lyrics'].iloc[i] = remove_punctuation(test_dataset['Lyrics'].iloc[i])

/var/folders/28/gj9cb4w95rb4sbjdp44jc_hh0000gn/T/ipykernel_20245/3746590888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Lyrics'].iloc[i] = remove_punctuation(test_dataset['Lyrics'].iloc[i])


In [ ]:
train_dataset.to_csv('train_genre.csv', index=False)
test_dataset.to_csv('test_genre.csv', index=False)

In [ ]:
dataset_Genre = train_dataset[['Lyrics','Genre']]

# HuggingFace Models

*luiz826/roberta-to-music-genre*

In [3]:
tokenizer = AutoTokenizer.from_pretrained("luiz826/roberta-to-music-genre")
pipe = pipeline("text-classification", model="luiz826/roberta-to-music-genre")

Some weights of the model checkpoint at luiz826/roberta-to-music-genre were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
dataset_Genre = train_dataset[['Lyrics','Genre']]

In [5]:
def tokenize_and_truncate(text, max_length):
    # Split the text into tokens (e.g., by whitespace)
    tokens = text.split()

    # Truncate tokens if the length exceeds max_length
    truncated_tokens = tokens[:max_length]

    # Join the truncated tokens back into a single string
    truncated_text = ' '.join(truncated_tokens)

    return truncated_text

In [6]:
dataset_Genre['luiz826/roberta-to-music-genre'] = None
for i in range(len(dataset_Genre)):
  try:
    dataset_Genre['luiz826/roberta-to-music-genre'].iloc[i] = pipe.predict(train_dataset['Lyrics'].iloc[i])[0]['label']
  except:
    continue

/var/folders/28/gj9cb4w95rb4sbjdp44jc_hh0000gn/T/ipykernel_29334/534057649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_Genre['luiz826/roberta-to-music-genre'] = None
/var/folders/28/gj9cb4w95rb4sbjdp44jc_hh0000gn/T/ipykernel_29334/534057649.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_Genre['luiz826/roberta-to-music-genre'].iloc[i] = pipe.predict(train_dataset['Lyrics'].iloc[i])[0]['label']
Token indices sequence length is longer than the specified maximum sequence length for this model (2032 > 2000). Running this sequence through t

*xlm-roberta-base-multilingual-text-genre-classifier*

In [ ]:
train_dataset = pd.read_csv("/content/drive/MyDrive/Genre_Dataset/train_genre.csv")
test_dataset = pd.read_csv("/content/drive/MyDrive/Genre_Dataset/test_genre.csv")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("classla/xlm-roberta-base-multilingual-text-genre-classifier")
model = AutoModelForSequenceClassification.from_pretrained("classla/xlm-roberta-base-multilingual-text-genre-classifier")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def tokenize_function(df):
    return tokenizer(df["Lyrics"], padding="max_length", truncation=True)

In [ ]:
tokenized_data = dict_df.map(tokenize_function)

AttributeError: 'dict' object has no attribute 'map'

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

KeyError: 102182

# Chord Generation

*ChatMusician*

In [ ]:
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 812.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
pip install accelerate

In [ ]:
name = "m-a-p/ChatMusician"
text = "[E Minor', 'A Minor', 'F Major', 'C Major']"
max_new_tokens = 500

In [ ]:
from transformers import pipeline

pipe = pipeline(model=name, model_kwargs= {"device_map": "auto", "load_in_8bit": True}, max_new_tokens=max_new_tokens)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ValueError: Could not load model m-a-p/ChatMusician with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py", line 280, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 563, in from_pretrained
    return model_class.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 3621, in from_pretrained
    hf_quantizer.validate_environment(device_map=device_map)
  File "/usr/local/lib/python3.10/dist-packages/transformers/qu

In [3]:
chord_dataset = pd.read_csv("/content/drive/MyDrive/Chord csv/Chord_Dataset.csv")

In [4]:
chord_dataset

,Unnamed: 0,Key,Chord Progression
0,0,E Major,"['A Major', 'C# Minor', 'E Major', 'E- Unknown']"
1,1,Db Major,"['Eb-minor triad', 'Gb-major triad', 'Ab-major..."
2,2,C Major,"['C Major', 'C Major', 'F# Unknown', 'F# Unkno..."
3,3,C Major,"['E Minor', 'A Minor', 'F Major', 'C Major']"
4,4,Db Major,"['F-minor triad', 'Bb-minor triad', 'Gb-major ..."
...,...,...,...
7351,7351,Gb Key,"['Abb-major triad', 'Cb-minor triad', 'Bbb-min..."
7352,7352,F Key,"['F Unknown', 'F Unknown', 'F Unknown', 'F Unk..."
7353,7353,F Key,"['F Unknown', 'F Unknown', 'F Unknown', 'F Unk..."
7354,7354,Db Key,"['Db-major triad', 'Ab-major triad', 'Gb-minor..."


In [ ]:
chord_dataset.drop(columns=['Unnamed: 0'])

,Key,Chord Progression
0,E Major,"['A Major', 'C# Minor', 'E Major', 'E- Unknown']"
1,Db Major,"['Eb-minor triad', 'Gb-major triad', 'Ab-major..."
2,C Major,"['C Major', 'C Major', 'F# Unknown', 'F# Unkno..."
3,C Major,"['E Minor', 'A Minor', 'F Major', 'C Major']"
4,Db Major,"['F-minor triad', 'Bb-minor triad', 'Gb-major ..."
...,...,...
7351,Gb Key,"['Abb-major triad', 'Cb-minor triad', 'Bbb-min..."
7352,F Key,"['F Unknown', 'F Unknown', 'F Unknown', 'F Unk..."
7353,F Key,"['F Unknown', 'F Unknown', 'F Unknown', 'F Unk..."
7354,Db Key,"['Db-major triad', 'Ab-major triad', 'Gb-minor..."


In [ ]:
chord_dataset['Chord Progression'][100]

"['B- Major', 'E- Major', 'F Major', 'E- Major']"

In [ ]:
pipe.predict('Eb-minor triad Gb-major triad Ab-major triad Ab-major triad')

[{'generated_text': 'Eb-minor triad Gb-major triad Ab-major triad Ab-major triad Bb-major triad Bb-major triad C-major triad C-major'}]

In [ ]:
pipe.predict('C Major G Major A Major triad F Major')

[{'generated_text': 'C Major G Major A Major triad F Major G Major A Major triad F Major G Major A Major triad F Major G Major A Major'}]

In [ ]:
pipe.predict('C Minor F Major B- Major B- Major')

[{'generated_text': 'C Minor F Major B- Major B- Major C- Major C- Major D- Major D- Major E- Major E- Major F-'}]

In [ ]:
pipe.predict('Ab-major triad Eb-major triad F-minor triad Eb-major triad')

[{'generated_text': 'Ab-major triad Eb-major triad F-minor triad Eb-major triad F-major triad G-major triad F-major triad G-minor tri'}]

In [ ]:
pipe.predict('B- Major E- Major F Major E- Major')

[{'generated_text': 'B- Major E- Major F Major E- Major F Major G Major E- Major F Major G Major A Major E- Major F Major G Major'}]

*musiclang/text-chord-predictor*

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="musiclang/text-chord-predictor")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
pipe.predict('START Ebm GbM AbM AbM')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1137: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'START Ebm GbM AbM AbM AbM AbM AbM AbM AbM AbM AbM AbM AbM AbM AbM AbM AbM AbM AbM'}]

In [ ]:
pipe.predict('START CM GM AM FM')

[{'generated_text': 'START CM GM AM FM GM GM GM GM GM GM GM GM GM GM GM GM GM GM GM'}]

In [ ]:
pipe.predict('START BM EM FM EM')

[{'generated_text': 'START BM EM FM EM EM EM EM EM EM EM EM EM EM EM EM EM EM EM EM'}]

References:

1. https://www.linkedin.com/pulse/music-genre-classification-magetech/
2. https://blog.enterprisedna.co/python-remove-punctuation-from-string/
3. https://www.kaggle.com/code/wfaria/midi-music-data-extraction-using-music21#MIDI-Processing
4. https://web.mit.edu/music21/doc/moduleReference/moduleRoman.html
5. https://ezmonyi.github.io/ChatMusician/
6. https://colab.research.google.com/drive/1qOjXfQIAULfKvZqwCen8-MoWKGdSatZ4#scrollTo=JM1NgY4B5Mx2

